In [1]:
] activate ..


  Activating environment at `e:\Pythonfiles\JP\ML\Project.toml`


In [2]:
using DifferentialEquations
using LinearAlgebra, DiffEqSensitivity, Optim
using Flux: flatten, params
using DiffEqFlux, Flux
using Plots
using Flux: train!
using NNlib

# Chapter 1. Dense()

One need to learn array first

## 1.1 Dense(1,1)

Define, input, weight, loss, opt, train!

Generate Data

In [ ]:
function l(x)
    2x+1
end
x = Array(0:0.01:3)'
y = l.(x)

Input should be  a Column vector. short to say, input should be a column tensor.  input default is row. output is column tensor

weight, bias, params.

In [ ]:
test_nn = Dense(1,1) # define layer
test_nn.weight #  weight output
test_nn.bias # bias output
p0 = Float64.(initial_params(test_nn)) # initial params
params(test_nn)


One can destructure the parameters of Chain, and redefine it and then restructure. For example, choose a new initializer. One can define initializer in layer.

For more details, can be seen in Flux src/utils

In [ ]:
model = Chain(Dense(5,1,relu,init = Flux.glorot_uniform),Dense(1,2))
θ, re = Flux.destructure(model)
θ = Flux.glorot_uniform(10)
re(θ)

In [ ]:
# test_nn(0.1) # input a constant. error
test_nn([0.1]) # no error
test_nn([0.1 0.2])
#test_nn(0:0.1:2) # error,\
x0=Array(0:1:2)' # 0:0.1:2 is a list, should be converted to array, and transpose
x0 =[0 1 2]
#x0==[0 1 2]
test_nn(x0) # true

Define opt loss and train! If x y satisfies column principle. loss should not be broadcasted.

data should be couple of [(x,y)] x should be column.

In [ ]:
loss(a,b) = Flux.Losses.mse(test_nn(a), b)
loss(x,y)
opt=Descent(0.1)
train!(loss, params(test_nn), [(x,y)], opt)
loss(x,y)

for epoch in 1:1000
    train!(loss, params(test_nn), [(x,y)], opt)
end
params(test_nn)
loss(x,y)
plot(x', test_nn(x)')

you can try different:
* parameter initializer
* layers: hidden layer, activation function
* loss function
* opt
* batch

## 1.2 Dense(2,1)


Generate Data

In [ ]:
function ff(x,y)
    x*y
end
x1 = Array(0:0.01:3)'# should be column
y1 = Array(4:0.01:7)'
z1 = ff.(x1,y1)
size(z1)
test_nn1= Dense(2,1)
xx = [x1;y1] # 
println(size(xx))
test_nn1(xx)
#[test_nn1([i[1],i[2]])[1] for i=zip(xx[1], xx[2])]
loss1(a,b)=Flux.Losses.mae(test_nn1(a), b)
loss1(xx,z1)

Train

In [ ]:
opt=ADAM(0.01)

train!(loss1, params(test_nn1), [(xx,z1)], opt)
println(loss1(xx,z1))

for epoch in 1:1000
    train!(loss1, params(test_nn1), [(xx,z1)], opt)
end
params(test_nn1)
println(loss1(xx,z1))
scatter(z1')
plot!(test_nn1(xx)')


loss: mse, mae, msle, crossentrophy

opt: Descent, ADAM(0.01), Momentum, RMSProp


# Chapter 2. Multilayer

layers including:

* function
* DNN
* RNN
* other architechture

Generate Data and define model

In [ ]:
# 1. Gennerate Data
x = Array(-2π:0.01:2π)'
function f(x)
    x*(1-x)+ x^3
end
data = sin.(x)
#dnn_model = Chain(Dense(1, 1), x-> cos.(x))
#dnn_model = Chain(Dense(1, 128, relu), Dense(128, 1), x-> cos.(x))
dnn_model = Chain(Dense(1, 10, swish), Dense(10, 100,swish),Dense(100, 10, swish), Dense(10, 1))
# dnn_model = Chain(Dense(1, 32, relu), Dense(32, 1, tanh))
data .- dnn_model(x)
loss2(a,b) = Flux.Losses.mae(dnn_model(x), data)
opt=ADAM(0.02)
println(loss2(x, data))
train!(loss2, params(dnn_model), [(x,data)], opt)
println(loss2(x, data))

for epoch in 1:5000
    train!(loss2, params(dnn_model), [(x,data)], opt)
end
println(loss2(x, data))
y = Array(-2π:0.1:4π)'
scatter(sin.(y)')
plot!(dnn_model(y)')

In [ ]:
model_fun = Chain(Dense(2,2,relu), x -> [sum(x)], Dense(1,1))
model_fun([1.0,2.0])

In [ ]:
# 1. Gennerate Data
x = Array(0:0.01:1)'
function f(x)
    0.2x+0.4x^2+0.3x*sin(15x)+0.05cos(50x)
end
data = f.(x)
#dnn_model = Chain(Dense(1, 1), x-> cos.(x))
#dnn_model = Chain(Dense(1, 128, relu), Dense(128, 1), x-> cos.(x))
dnn_model = Chain(Dense(1, 10, sigmoid), Dense(10, 100,swish),Dense(100, 10, swish), Dense(10, 1))
# dnn_model = Chain(Dense(1, 32, relu), Dense(32, 1, tanh))
data .- dnn_model(x)
loss2(a,b) = Flux.Losses.mae(dnn_model(x), data)
opt=ADAM(0.02)
println(loss2(x, data))
train!(loss2, params(dnn_model), [(x,data)], opt)
println(loss2(x, data))

for epoch in 1:5000
    train!(loss2, params(dnn_model), [(x,data)], opt)
end
println(loss2(x, data))
y = Array(0:0.1:1.3)'
scatter(f.(y)')
plot!(dnn_model(y)')

# train the model

In [ ]:
"""
loss2(a,b) = Flux.Losses.mae(dnn_model(x), data)
opt=ADAM(0.01)
println(loss2(x, data))
train!(loss2, params(dnn_model), [(x,data)], opt)
println(loss2(x, data))

for epoch in 1:1000
    train!(loss2, params(dnn_model), [(x,data)], opt)
end
println(loss2(x, data))
y = Array(-2π:0.1:4π)'
scatter(sin.(y)')
plot!(dnn_model(y)')
"""

# Chapter 3 Data handling, callbacks and save and reload

In [ ]:
# 1. Gennerate Data
x = Array(-2π:0.01:2π)'
function f(x)
    x*(1-x)+ x^3
end
data = sin.(x)
#dnn_model = Chain(Dense(1, 1), x-> cos.(x))
#dnn_model = Chain(Dense(1, 128, relu), Dense(128, 1), x-> cos.(x))
dnn_model = Chain(Dense(1, 10, swish), Dense(10, 100,swish),Dense(100, 10, swish), Dense(10, 1))
# dnn_model = Chain(Dense(1, 32, relu), Dense(32, 1, tanh))
data .- dnn_model(x)
loss2(a,b) = Flux.Losses.mae(dnn_model(x), data)
opt=ADAM(0.02)
println(loss2(x, data))
train!(loss2, params(dnn_model), [(x,data)], opt)
println(loss2(x, data))



for epoch in 1:5000
    train!(loss2, params(dnn_model), [(x,data)], opt)
    if epoch%500==0
    println(loss2(x, data))
    end
end
println(loss2(x, data))
y = Array(-2π:0.1:4π)'
scatter(sin.(y)')
plot!(dnn_model(y)')



Save and load

In [ ]:
using BSON: @save
using BSON: @load
@save "sin_learn_model.bason" dnn_model
@load "sin_learn_model.bason" dnn_model
dnn_model(x)

# Chapter 4 ODE as an layer
the essence is regarding ode solution as a function.

f(u_0, p)

Input can be u_0 or p. and output of former layer can be input of ode (parameter or initialvalue)

the output can be timeseries or end value. Format should be row

In [ ]:
function SIR!(du,u,p,t)
    Λ, d, β, γ = p
    S, I = u
    du[1] = Λ - β*S*I - d*S
    du[2] = β*S*I - (d+γ)*I
end
u_0 = [10.0, 0.1]
p_data = [1,0.1,0.3,0.1]
tspan_data = (0.0, 2.0)
function solution_SIR(p)
    prob_data = ODEProblem(SIR!,p,tspan_data,p_data)
    data_solve = solve(prob_data, Tsit5(),abstol=1e-12, reltol=1e-12, saveat = 0.1)
end
println(solution_SIR(u_0)[:,end]')

function f(x)
    2*x+1
end
#coupled_model = Chain(Dense(2, 21, relu), p -> solution_SIR(p)[1,:], Dense(21,5, tanh),Dense(5, 1))
coupled_model = Chain(Dense(2, 2, relu), p -> solution_SIR(p)[1,:], x -> x[2:end], Dense(20,5, tanh),Dense(5, 1))
#coupled_model = FastChain(FastDense(2, 2, relu),x -> f.(x), p -> solution_SIR(p)[:,end]', FastDense(2,5, tanh),FastDense(5, 1))
#coupled_model = Chain(Dense(2, 2, relu),x -> f.(x), p -> solution_SIR(p)[:,end], Dense(2,5, tanh),Dense(5, 1))

u_1 = [5, 0.1]
#p_init = Float64.(initial_params(coupled_model))
println(coupled_model(u_1))



# Chapter 4 DiffEqFlux

DiffEqFlux (FastChain) 和 Flux (Chain) 的 区别在于。 Flux中参数都有默认值，除非重新用Flux.params()申明是参数。所以里边默认值，想让他变成参数，只用在Flux.train!()参数中，用Flux.params()包起来。

DiffEqFlux里边算子是算子，参数是参数，可以理解为destructure中的re。各有各的好处

## 4.1 FastDense(1,1)

Note here that FastDense is only operator and eat (data, parameter). Dense eat data. FastDense can eat constant.

In [ ]:
function l(x)
    2x+1
end
x = Array(0:0.01:3)'
y = l.(x)

test_nn = FastDense(1,1) # define layer
params(test_nn) #  weight output
#test_nn.bias # bias output
p0 = Float64.(initial_params(test_nn)) # initial params
params(test_nn)
test_nn(x,p0)

In [ ]:
# test_nn(0.1) # input a constant. error
test_nn(0.1,p0) # no error
test_nn([0.1 0.2],p0)
#test_nn(0:0.1:2) # error,\
x0=Array(0:1:2)' # 0:0.1:2 is a list, should be converted to array, and transpose
x0 =[0 1 2]
#x0==[0 1 2]
test_nn(x0,p0) # true

In [ ]:
model_test = FastChain(FastDense(2,1,swish),FastDense(1,2))
model_test1 = FastChain(FastDense(2,1,swish),FastDense(1,2))
model_test2 = FastChain(model_test, model_test1)
u0 = [0.1,0.2]
p = Float64.(initial_params(model_test1))
p_0 = [0.1,0.2,0.3,0.4,0.5,0.6,0.7]
print(p)
model_test1(u0,[p;p_0])

In [ ]:
function train(θ)
    test_nn(x,θ)
end


# loss, keep the same

function loss(θ)
    pred = train(θ)
    sum(abs2, (y .- pred)), pred # + 1e-5*sum(sum.(abs, params(ann)))
end

# callback, keep  the same
const losses = []
callback(θ,l,pred) = begin
    push!(losses, l)
    if length(losses)%50==0
        println(losses[end])
    end
    false
end

res = DiffEqFlux.sciml_train(loss, p0, ADAM(0.01), cb=callback, maxiters = 1000)
#res2_node = DiffEqFlux.sciml_train(loss, res1_node.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 1000)
loss(res.minimizer)
plot(x', test_nn(x,res.minimizer)')

## 4.2. Multilayer

function layer should be (x,p) -> f.(x, p). x is output of former layer, p is parameter.

In [ ]:
# 1. Gennerate Data
x = Array(-2π:0.01:2π)'
function f(x)
    x*(1-x)+ x^3
end
data = sin.(x)
#dnn_model = Chain(Dense(1, 1), x-> cos.(x))
#dnn_model = Chain(Dense(1, 128, relu), Dense(128, 1), x-> cos.(x))
dnn_model = FastChain(FastDense(1, 10, swish), FastDense(10, 100,swish),FastDense(100, 10, swish), FastDense(10, 1), (x,p)->cos.(x))
# dnn_model = Chain(Dense(1, 32, relu), Dense(32, 1, tanh))
p = Float64.(initial_params(dnn_model)) 

function train(θ)
    dnn_model(x,θ)
end


# loss, keep the same

function loss(θ)
    pred = train(θ)
    sum(abs2, (data .- pred)), pred # + 1e-5*sum(sum.(abs, params(ann)))
end

# callback, keep  the same
const losses = []
callback(θ,l,pred) = begin
    push!(losses, l)
    if length(losses)%50==0
        println(losses[end])
    end
    false
end

res = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 1000)
#res2_node = DiffEqFlux.sciml_train(loss, res1_node.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 1000)
loss(res.minimizer)
y = Array(-2π:0.1:4π)'
plot(y', dnn_model(y,res.minimizer)')
scatter!(y', sin.(y)')


## 5.3 NeuralODE as a layer

### 5.3.1 method one， solution map as a function

remember that for DiffEqFlux, function should be (x,p) -> f(x).

Thus, ode as a function in this method. One can not estimate parameters but can use parameter as output of upper layer.

In [ ]:
function SIR!(du,u,p,t)
    Λ, d, β, γ = p
    S, I = u
    du[1] = Λ - β*S*I - d*S
    du[2] = β*S*I - (d+γ)*I
end
u_0 = [10.0, 0.1]
p_data = [1,0.1,0.3,0.1]
tspan_data = (0.0, 2.0)
function solution_SIR(u)
    prob_data = ODEProblem(SIR!,u,tspan_data,p_data)
    data_solve = solve(prob_data, Tsit5(),abstol=1e-12, reltol=1e-12, saveat = 0.1)
end
println(solution_SIR(u_0)[:,end]')

function f(x)
    2*x+1
end
#coupled_model = Chain(Dense(2, 21, relu), p -> solution_SIR(p)[1,:], Dense(21,5, tanh),Dense(5, 1))
coupled_model = FastChain(FastDense(2, 2, relu), (x, p) -> solution_SIR(x)[1,:], (x, p) -> x[2:end], FastDense(20,5, tanh),FastDense(5, 1))
#coupled_model = FastChain(FastDense(2, 2, relu),x -> f.(x), p -> solution_SIR(p)[:,end]', FastDense(2,5, tanh),FastDense(5, 1))
#coupled_model = Chain(Dense(2, 2, relu),x -> f.(x), p -> solution_SIR(p)[:,end], Dense(2,5, tanh),Dense(5, 1))

u_1 = [5, 0.1]
p_init = Float64.(initial_params(coupled_model))
println(coupled_model(u_1,p_init))

### 5.3.2 Method two: Neural ODE

for this kind, former layer input should be initial value of neuralode, parameters should be layer parameter. output are sequence.

NeuralODE

In [ ]:

u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[1], tspan[2], length = datasize)

function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u.^3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))

dudt2 = FastChain((x, p) -> x.^3,
                  FastDense(2, 50, tanh),
                  FastDense(50, 2))
model1 = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

#model1 = FastChain(FastDense(2,2,swish), nn_ode, FastDense(21,1,relu))
#p1 = Float64.(initial_params(model1))
model1(u0,model1.p)



In [ ]:
function SIR!(u,p)
    Λ, d, β, γ = p
    S, I = u
    du[1] = Λ - β*S*I - d*S
    du[2] = β*S*I - (d+γ)*I
    [du[1],du[2]]
end
u_0 = [10.0, 0.1]
p_data = [1,0.1,0.3,0.1]
tspan_data = (0.0f0, 2.0f0)

chain_1 = FastChain(FastDense(2,21,swish), FastDense(21,2,relu))

model2 = NeuralODE(chain_1, tspan_data, Tsit5(), saveat = 0.1)

#model1 = FastChain(FastDense(2,2,swish), nn_ode, FastDense(21,1,relu))
#p1 = Float64.(initial_params(model1))
model2(u_0,model2.p)

# Chapter 6 DiffEqFlux Basis

## 6.1 Autodifferentialation

Zygote

ReverseDiff

ForwardDiff

In [ ]:
using DiffEqSensitivity, OrdinaryDiffEq, Zygote

function fiip(du,u,p,t)
  du[1] = dx = p[1]*u[1] - p[2]*u[1]*u[2]
  du[2] = dy = -p[3]*u[2] + p[4]*u[1]*u[2]
end
p = [1.5,1.0,3.0,1.0]; u0 = [1.0;1.0]
prob = ODEProblem(fiip,u0,(0.0,10.0),p)
sol = solve(prob,Tsit5())
loss(u0,p) = sum(solve(prob,Tsit5(),u0=u0,p=p,saveat=0.1))
du01,dp1 = Zygote.gradient(loss,u0,p)

Also adjoint method

ReverseDiffAdjoint

QuadratureAdjoint

InterpolatingAdjoint

The following algorithm choices exist for sensealg. See the sensitivity mathematics page for more details on the definition of the methods.

* ForwardSensitivity(;ADKwargs...): An implementation of continuous forward sensitivity analysis for propagating derivatives by solving the   extended ODE. Only supports ODEs.
* ForwardDiffSensitivity(;chunk_size=0,convert_tspan=true): An implementation of discrete forward sensitivity analysis through ForwardDiff.jl. This algorithm can differentiate code with callbacks when convert_tspan=true, but will be faster when convert_tspan=false.
* BacksolveAdjoint(;checkpointing=true,ADKwargs...): An implementation of adjoint sensitivity analysis using a backwards solution of the ODE. By default this algorithm will use the values from the forward pass to perturb the backwards solution to the correct spot, allowing reduced memory with stabilization. Only supports ODEs and SDEs.
* InterpolatingAdjoint(;checkpointing=false;ADKwargs...): The default. An implementation of adjoint sensitivity analysis which uses the interpolation of the forward solution for the reverse solve vector-Jacobian products. By default it requires a dense solution of the forward pass and will internally ignore saving arguments during the gradient calculation. When checkpointing is enabled it will only require the memory to interpolate between checkpoints. Only supports ODEs and SDEs.
* QuadratureAdjoint(;abstol=1e-6,reltol=1e-3,compile=false,ADKwargs...): An implementation of adjoint sensitivity analysis which develops a full continuous solution of the reverse solve in order to perform a post-ODE quadrature. This method requires the the dense solution and will ignore saving arguments during the gradient calculation. The tolerances in the constructor control the inner quadrature. The inner quadrature uses a ReverseDiff vjp if autojacvec, and compile=false by default but can compile the tape under the same circumstances as ReverseDiffVJP. Only supports ODEs.
* ReverseDiffAdjoint(): An implementation of discrete adjoint sensitivity analysis using the ReverseDiff.jl tracing-based AD. Supports in-place functions through an Array of Structs formulation, and supports out of place through struct of arrays.
* TrackerAdjoint(): An implementation of discrete adjoint sensitivity analysis using the Tracker.jl tracing-based AD. Supports in-place functions through an Array of Structs formulation, and supports out of place through struct of arrays.
* ZygoteAdjoint(): An implementation of discrete adjoint sensitivity analysis using the Zygote.jl source-to-source AD directly on the differential equation solver. Currently fails.
* SensitivityADPassThrough(): Ignores all adjoint definitions and proceeds to do standard AD through the solve functions.
* ForwardLSS(), AdjointLSS(), and NILSS(nseg,nstep): Implementation of shadowing methods for chaotic systems with a long-time averaged objective. See the sensitivity analysis for chaotic systems (shadowing methods) section for more details.


The ReverseDiffAdjoint(), TrackerAdjoint(), ZygoteAdjoint(), and SensitivityADPassThrough() algorithms all offer differentiate-through-the-solver adjoints, each based on their respective automatic differentiation packages. If you're not sure which to use, ReverseDiffAdjoint() is generally a stable and performant best if using the CPU, while TrackerAdjoint() is required if you need GPUs. Note that SensitivityADPassThrough() is more or less an internal implementation detail. For example, ReverseDiffAdjoint() is implemented by invoking ReverseDiff's AD functionality on solve(...; sensealg=SensitivityADPassThrough()).

In [ ]:
function sum_of_solution(u0,p)
    _prob = remake(prob,u0=u0,p=p)
    sum(solve(_prob,Tsit5(),rtol=1e-6,atol=1e-6,saveat=0.1,sensealg=QuadratureAdjoint()))
end
du01,dp1 = Zygote.gradient(sum_of_solution,u0,p)

# 6.2 Train ODE

## DiffEqFlux.sciml_train
—
Function
sciml_train

* Unconstrained Optimization
```
function sciml_train(loss, _θ, opt = DEFAULT_OPT, adtype = DEFAULT_AD,
                     _data = DEFAULT_DATA, args...;
                     cb = (args...) -> false, maxiters = get_maxiters(data),
                     kwargs...)
```
* Box Constrained Optimization
```
function sciml_train(loss, θ, opt = DEFAULT_OPT, adtype = DEFAULT_AD,
                     data = DEFAULT_DATA, args...;
                     lower_bounds, upper_bounds,
                     cb = (args...) -> (false), maxiters = get_maxiters(data),
                     kwargs...)
```
* Optimizer Choices and Arguments (GalacticOptim.jl)

For a full definition of the allowed optimizers and arguments, please see the GalacticOptim.jl documentation. As sciml_train is an interface over GalacticOptim.jl, all of its optimizers and arguments can be used from here.

* Loss Functions and Callbacks

Loss functions in sciml_train treat the first returned value as the return. For example, if one returns (1.0, [2.0]), then the value the optimizer will see is 1.0. The other values are passed to the callback function. The callback function is cb(p, args...) where the arguments are the extra returns from the loss. This allows for reusing instead of recalculating. The callback function must return a boolean where if true, then the optimizer will prematurely end the optimization. It is called after every successful step, something that is defined in an optimizer-dependent manner.

* Default AD Choice

The current default AD choice is dependent on the number of parameters, where for <100 parameters ForwardDiff.jl is used, otherwise Zygote.jl is used. More refinements to the techniques are planned.

* Default Optimizer Choice

By default, if the loss function is deterministic than an optimizer chain of ADAM -> BFGS is used, otherwise ADAM is used (and a choice of maxiters is required).

In [ ]:
using DifferentialEquations, DiffEqFlux, Plots

function lotka_volterra!(du, u, p, t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

# Initial condition
u0 = [1.0, 1.0]

# Simulation interval and intermediary points
tspan = (0.0, 10.0)
tsteps = 0.0:0.1:10.0

# LV equation parameter. p = [α, β, δ, γ]
p = [1.5, 1.0, 3.0, 1.0]

# Setup the ODE problem, then solve
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob, Tsit5())

# Plot the solution
using Plots
plot(sol)

function loss(p)
  sol = solve(prob, Tsit5(), p=p, saveat = tsteps)
  loss = sum(abs2, sol.-1)
  return loss, sol
end
"""
callback = function (p, l, pred)
  display(l)
  plt = plot(pred, ylim = (0, 6))
  display(plt)
  # Tell sciml_train to not halt the optimization. If return true, then
  # optimization stops.
  return false
end
"""
result_ode = DiffEqFlux.sciml_train(loss, p,
                                    #cb = callback,
                                    lower_bounds = (1, 0, 2.0, 0),
                                    upper_bounds = (5, 4, 4.0, 3),
                                    maxiters = 100)

In [ ]:
using DiffEqFlux, DifferentialEquations, LinearAlgebra
k, α, β, γ = 1, 0.1, 0.2, 0.3
tspan = (0.0,10.0)

function dxdt_train(du,u,p,t)
  du[1] = u[2]
  du[2] = -k*u[1] - α*u[1]^3 - β*u[2] - γ*u[2]^3
end

u0 = [1.0,0.0]
ts = collect(0.0:0.1:tspan[2])
prob_train = ODEProblem{true}(dxdt_train,u0,tspan,p=nothing)
data_train = Array(solve(prob_train,Tsit5(),saveat=ts))


A = [LegendreBasis(10), LegendreBasis(10)]
nn = TensorLayer(A, 1)
f = x -> min(30one(x),x)

function dxdt_pred(du,u,p,t)
  du[1] = u[2]
  du[2] = -p[1]*u[1] - p[2]*u[2] + f.(nn(u,p[3:end])[1])
end

α = zeros(102)

prob_pred = ODEProblem{true}(dxdt_pred,u0,tspan,p=nothing)

function predict_adjoint(θ)
    x = Array(solve(prob_pred,Tsit5(),p=θ,saveat=ts))
end
  
function loss_adjoint(θ)
    x = predict_adjoint(θ)
    loss = sum(norm.(x - data_train))
    return loss
end
  
function cb(θ,l)
@show θ, l
return false
end

res1 = DiffEqFlux.sciml_train(loss_adjoint, α, ADAM(0.05), cb = cb, maxiters = 150)
res2 = DiffEqFlux.sciml_train(loss_adjoint, res1.u, ADAM(0.001), cb = cb,maxiters = 150)
opt = res2.u

using Plots
data_pred = predict_adjoint(opt)
plot(ts, data_train[1,:], label = "X (ODE)")
plot!(ts, data_train[2,:], label = "V (ODE)")
plot!(ts, data_pred[1,:], label = "X (NN)")
plot!(ts, data_pred[2,:],label = "V (NN)")

## UDE 和 ODE 参数花式共估计
本例子中，ODE的初值，参数也可以估计。

In [ ]:
using DiffEqFlux, DifferentialEquations, Plots

u0 = 1.1f0
tspan = (0.0f0, 25.0f0)
tsteps = 0.0f0:1.0:25.0f0

model_univ = FastChain(FastDense(2, 16, tanh),
                       FastDense(16, 16, tanh),
                       FastDense(16, 1))

# The model weights are destructured into a vector of parameters
p_model = initial_params(model_univ)
n_weights = length(p_model)

# Parameters of the second equation (linear dynamics)
p_system = Float32[0.5, -0.5]

p_all = [p_model; p_system]
θ = Float32[u0; p_all]

function dudt_univ!(du, u, p, t)
    # Destructure the parameters
    model_weights = p[1:n_weights]
    α = p[end - 1]
    β = p[end]

    # The neural network outputs a control taken by the system
    # The system then produces an output
    model_control, system_output = u

    # Dynamics of the control and system
    dmodel_control = model_univ(u, model_weights)[1]
    dsystem_output = α*system_output + β*model_control

    # Update in place
    du[1] = dmodel_control
    du[2] = dsystem_output
end

prob_univ = ODEProblem(dudt_univ!, [0f0, u0], tspan, p_all)
sol_univ = solve(prob_univ, Tsit5(),abstol = 1e-8, reltol = 1e-6)

function predict_univ(θ)
  return Array(solve(prob_univ, Tsit5(), u0=[0f0, θ[1]], p=θ[2:end],
                              sensealg = InterpolatingAdjoint(autojacvec=ReverseDiffVJP(true)),
                              saveat = tsteps))
end

loss_univ(x) = sum(abs2, predict_univ(x)[2,:] .- 1)
#l = loss_univ(θ)

list_plots = []
iter = 0
"""
function cb(θ,l)
  @show θ, l
  return false
end
callback = function (θ, l)
  global list_plots, iter

  if iter == 0
    list_plots = []
  end
  iter += 1

  if iter%50==0

  println(l)

  plt = plot(predict_univ(θ)', ylim = (0, 6))
  push!(list_plots, plt)
  display(plt)
  end
  return false
end
"""
result_univ = DiffEqFlux.sciml_train(loss_univ, θ)
                                     #cb = cb)
plot(predict_univ(result_univ.minimizer)', ylim = (0, 6))

# Chapter 7. Tensor Layer


In [99]:
# 1. Gennerate Data from sin(x)
x = Array(-2π:0.1:2π)'
data = sin.(x)

# 2. learn by dnn with a hidden TensorLayer
function dnn_output(θ,v)
    tensor_layer = TensorLayer([SinBasis(2)], 1)
    dnn_model = FastChain(FastDense(1,1),(y,p)-> tensor_layer(y,θ[1:2])[1], FastDense(1, 1))
    value = []
    for i = v
    push!(value,dnn_model(i,θ[3:end])[1])
    end
    Array(value)
end
function train(θ)
    dnn_output(θ,x)
end
p = randn(6)
println(train(p))  # output a vector.
function loss(θ)
    pred = train(θ)
    sum(abs2, (data .- pred))
end
println(loss(p)) # 29790
res = DiffEqFlux.sciml_train(loss, p, ADAM(0.05), maxiters = 2000)
println(loss(res.minimizer)) # 7916

Any[0.024067914020447057, 0.0770612827775583, 0.12570722627111697, 0.16970812760320353, 0.20879077934659707, 0.24270796726385768, 0.2712399092445992, 0.2941955412443673, 0.31141364287290574, 0.3227637961858578, 0.32814717217679545, 0.3274971404406819, 0.3207796984798439, 0.3079937181436583, 0.2891710077275931, 0.2643761893001775, 0.2337063918719533, 0.1972907620625799, 0.1552897949551093, 0.1078944888441371, 0.05532532858135519, -0.002168896807748122, -0.0643124356241015, -0.13080406659944144, -0.2013187352338539, -0.2755093241844586, -0.35300854776427015, -0.43343095990751246, -0.5163750643151953, -0.6014255149161504, -0.6881553942673373, -0.7761285570758356, -0.8649020256562162, -0.9540284238428438, -1.0430584356589234, -1.1315432749038752, -1.2190371517588399, -1.3050997225269403, -1.3892985087204945, -1.4712112718809232, -1.5504283307677742, -1.6265548078796828, -1.6992127926701999, -1.768043409293211, -1.8327087772532353, -1.8928938539423146, -1.9483081487140173, -1.99868729887240

LoadError: Cannot convert Adjoint{Any, Vector{Any}} to series data for plotting

In [ ]:
using DiffEqFlux, DifferentialEquations, LinearAlgebra
k, α, β, γ = 1, 0.1, 0.2, 0.3
tspan = (0.0,10.0)

function dxdt_train(du,u,p,t)
  du[1] = u[2]
  du[2] = -k*u[1] - α*u[1]^3 - β*u[2] - γ*u[2]^3
end

u0 = [1.0,0.0]
ts = collect(0.0:0.1:tspan[2])
prob_train = ODEProblem{true}(dxdt_train,u0,tspan,p=nothing)
data_train = Array(solve(prob_train,Tsit5(),saveat=ts))

A = [LegendreBasis(10), LegendreBasis(10)]
nn = TensorLayer(A, 1)

f = x -> min(30one(x),x)

function dxdt_pred(du,u,p,t)
  du[1] = u[2]
  du[2] = -p[1]*u[1] - p[2]*u[2] + f(nn(u,p[3:end])[1])
end

α = zeros(102)

prob_pred = ODEProblem{true}(dxdt_pred,u0,tspan,p=nothing)

function predict_adjoint(θ)
    x = Array(solve(prob_pred,Tsit5(),p=θ,saveat=ts))
end

function loss_adjoint(θ)
x = predict_adjoint(θ)
loss = sum(norm.(x - data_train))
return loss
end

function cb(θ,l)
@show θ, l
return false
end
res1 = DiffEqFlux.sciml_train(loss_adjoint, α, ADAM(0.05), cb = cb, maxiters = 150)
res2 = DiffEqFlux.sciml_train(loss_adjoint, res1.u, ADAM(0.001), cb = cb,maxiters = 150)
opt = res2.u
using Plots
data_pred = predict_adjoint(opt)
plot(ts, data_train[1,:], label = "X (ODE)")
plot!(ts, data_train[2,:], label = "V (ODE)")
plot!(ts, data_pred[1,:], label = "X (NN)")
plot!(ts, data_pred[2,:],label = "V (NN)")

In [45]:
using DiffEqFlux, DifferentialEquations, LinearAlgebra
B = [PolynomialBasis(2)]
nn0 = Dense(1,1)
nn1 = TensorLayer(B, 1)
nn2 = FastDense(1,1)
u = [1 2 3]
println(nn0(u)) # Float32[0.95563 1.91126 2.86689]
println(nn1(u,zeros(2))) # [-0.9934653167126005]
println(nn2(u,initial_params(nn2))) # Float32[-0.7425173 -1.4850346 -2.227552]


Float32[0.88244027 1.7648805 2.6473207]
[0.0]
Float32[-0.967725 -1.93545 -2.9031749]


In [50]:
p_0 = 0.1
nn2 = FastChain(FastDense(1,1), (x, p)->p_0*x)
p_1 = initial_params(nn2)
u = [1 2 3]
print(nn2(u,p_1))



## input a vector
B = [PolynomialBasis(2),SinBasis(2)]
nn_tensor = TensorLayer(B, 1)
p_tensor = ones(4)
function ff(x)
    a = nn_tensor(x,p_tensor)[1]
    a
end
nn_full = FastChain(FastDense(1,2),(x,p)-> ff(x))
uu = [1 2 3]
p_net = initial_params(nn_full)
value = []
for u=uu
push!(value,nn_full(u, p_net))
end
value

[-0.16022876501083375 -0.3204575300216675 -0.4806862831115723]

3-element Vector{Any}:
 -0.013367999035494904
 -1.781744184024319
 -2.2373552657361557

In [49]:
p_tensor = zeros(2)
nn = TensorLayer([SinBasis(2)],1)
function f(x)
nn(x, p_tensor)[1]
end
dnn_model = FastChain(FastDense(1,1), (x,p) -> f.(x))
u = [1 2 3]
dnn_model(u,initial_params(dnn_model))

1×3 Matrix{Float64}:
 0.0  0.0  0.0

In [65]:
p_tensor = zeros(4)
nn = TensorLayer([SinBasis(2),CosBasis(2)],1)
function f(x)
nn(x, p_tensor)[1]
end
dnn_model = FastChain(FastDense(1,2), (x,p) -> f(x))
u = [1 2]
dnn_model(u,initial_params(dnn_model))

0.0

In [79]:
nn = TensorLayer([SinBasis(2),CosBasis(2)],1)
nn_1 = Chain(Dense(1,2),nn)
nn_2 = Chain(Dense(1,2),Dense(2,1))
u = [1.0 2.0 3.0]
println(nn_1(u)) # [-0.02888989486945616]
println(nn_2(u)) # [-1.239922964687894 -2.479845929375788 -3.719768894063682]

[-0.5720381313787044]
[-1.3034050837082134 -2.606810167416427 -3.91021525112464]


In [77]:
nn = TensorLayer([SinBasis(2),CosBasis(2)],1)
p, re = Flux.destructure(Dense(2,1))#(nn)
nn_3 = FastChain(FastDense(1,2),re(p))
p_0 = initial_params(nn_3)
nn_3([1 2 3], p_0)

LoadError: MethodError: no method matching (::Dense{typeof(identity), Matrix{Float32}, Vector{Float32}})(::Matrix{Float32}, ::Vector{Float32})
[0mClosest candidates are:
[0m  (::Dense)(::AbstractVecOrMat{T} where T) at F:\Code\Julia\Julia-1.6.2\jpkgs\.julia\packages\Flux\Zz9RI\src\layers\basic.jl:146
[0m  (::Dense)(::AbstractArray) at F:\Code\Julia\Julia-1.6.2\jpkgs\.julia\packages\Flux\Zz9RI\src\layers\basic.jl:151

In [4]:
using DiffEqFlux, Flux
using Flux: flatten, params
x = Array(0:0.1:π)'
y = Array(0:0.1:π)'
f(a,b) = sin(a)+sin(b)
z = f.(x,y)
nn = TensorLayer([SinBasis(2),SinBasis(2)],1)
nn([1,1])


1-element Vector{Float64}:
 -1.3789745876764206

In [ ]:
# 1. Gennerate Data from Lorenz system
function lorenz(u,p,t)
    x, y, z = u

    ẋ = 10.0*(y - x)
    ẏ = x*(28.0-z) - y
    ż = x*y - (8/3)*z
    return [ẋ, ẏ, ż]
end

u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
dt = 0.1
prob = ODEProblem(lorenz,u0,tspan)
sol = solve(prob, Tsit5(), saveat = dt, progress = true)
data_withoutnois = Array(sol)
PolynomialBasis
# 2. learn by dnn with a hidden TensorLayer
function dnn_output(θ,v)
    tensor_layer = TensorLayer([SinBasis(2)], 1)
    dnn_model = FastChain(FastDense(1,1),(y,p)-> tensor_layer(y,θ[1:2])[1], FastDense(1, 1))
    value = []
    for i = v
    push!(value,dnn_model(i,θ[3:end])[1])
    end
    Array(value)
end
function train(θ)
    dnn_output(θ,x)
end
p = randn(6)
println(train(p))  # output a vector.
function loss(θ)
    pred = train(θ)
    sum(abs2, (data .- pred))
end
println(loss(p)) # 29790
res = DiffEqFlux.sciml_train(loss, p, ADAM(0.05), maxiters = 2000)
println(loss(res.minimizer)) # 7916